In [ ]:
import os
import json
import time
import openai

In [ ]:
# 1. Cấu hình API key (hoặc đặt biến môi trường OPENAI_API_KEY)
openai.api_key = os.getenv("OPENAI_API_KEY", "sk-...")

In [ ]:
# 2. Hàm gọi GPT để gán label cho một record
def label_options(record, model="gpt-4", temperature=0.0):
    prompt = f"""
Bạn là trợ lý AI, nhiệm vụ phân loại các phương án trả lời về thời lượng:
- Input là JSON có fields "context", "question", "options" (là list các string)
- Output chỉ trả về JSON list "labels" có cùng độ dài với "options", giá trị "yes" nếu option đúng mô tả thời lượng, ngược lại "no".

Ví dụ:
Input:
{{
  "context": "Tôi đang sửa chữa chiếc xe đạp bị hỏng.",
  "question": "Mất bao lâu để sửa chữa chiếc xe đạp?",
  "options": ["30 phút", "1 tháng", "10 phút", "2 giờ"]
}}
Output:
["yes", "no", "yes", "yes"]

Bây giờ phân loại cho record sau:
{json.dumps(record, ensure_ascii=False)}
"""
    resp = openai.ChatCompletion.create(
        model=model,
        messages=[{"role":"system","content":"Bạn là trợ lý AI chuyên gán label yes/no."},
                  {"role":"user","content":prompt}],
        temperature=temperature,
        max_tokens=100,
    )
    # GPT trả về content là dạng JSON list
    labels = json.loads(resp.choices[0].message.content.strip())
    return labels

In [ ]:
def main(input_path="public_test.txt", output_path="labeled_test.txt"):
    with open(input_path, "r", encoding="utf-8") as fin, \
         open(output_path, "w", encoding="utf-8") as fout:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            record = json.loads(line)
            # Gọi GPT gán labels
            try:
                labels = label_options(record)
            except Exception as e:
                print(f"Error when labeling QID={record.get('qid')}: {e}")
                # nếu lỗi, có thể thử chờ rồi gọi lại
                time.sleep(5)
                labels = label_options(record)
            # Gắn thêm trường labels
            record["labels"] = labels
            # Ghi ra file (mỗi dòng 1 JSON)
            fout.write(json.dumps(record, ensure_ascii=False) + "\n")
            # tránh call quá nhanh
            time.sleep(0.5)

In [ ]:
if __name__ == "__main__":
    main()
    print("Hoàn thành! Kết quả nằm ở labeled_test.txt")